# Learnings

* Create table using the API
* Upload data using the API
* Upload metadata using the API
* Delete table using the API

# Requirements

* Python packages:
    * `requests`
    * `pandas`
    * jupyter notebook, e.g. `notebook` or `jupyterlab`
* Get your OEP API token from the settings tab in your profile page    
* Skills:
    * use python with jupyter notebooks


In [ ]:
# install required packages with: pip install requests pandas
# import required packages
import json

import requests as req
import pandas as pd

In [ ]:
# TODO: explain schema/topic/sandbox
# TODO: explain randint

from random import randint
from getpass import getpass
from os import environ

topic = 'sandbox'
table = f'test_table_{randint(0, 100000)}'
token = environ.get("OEP_API_TOKEN") or getpass.getpass('Enter your OEP API token:')

# for read/write, we need to add authorization header
auth_headers = {'Authorization': 'Token %s' % token}
table_api_url = f"https://openenergy-platform.org/api/v0/schema/{topic}/tables/{table}/"

# Create table using the API

In [ ]:
# TODO: explain / link to data types
table_schema = {
    "columns": [
        # NOTE: first column should be numerical column named `id` .
        # Use `bigserial` if you want the database to create the re
        {"name": "id", "data_type": "bigserial", "primary_key": True},
        {"name": "name", "data_type": "varchar(18)", "is_nullable": False},
        {"name": "is_active", "data_type": "boolean"},
        {"name": "capacity_mw", "data_type": "float"},
        {"name": "installation_datetime_utc", "data_type": "datetime"},        
        {"name": "location", "data_type": "geometry"},
    ]
}

In [ ]:
# Creating table needs authentication headers
# Creating table needs table schema json data
res = req.put(table_api_url, json={"query": table_schema}, headers=auth_headers)

# raise Exception if request fails
if not res.ok:
    raise Exception(res.text)

# Upload data using the API

In [ ]:
# TODO: explain required data structure

# get example data (from csv)
df = pd.read_csv("https://gist.githubusercontent.com/wingechr/4b6a6301a37a2131c10b2eb412b288e9/raw/782c94009fe12486ffc7a232fd852fcda2e835bc/oep_example1.data.csv")

# optionally: clean up nan values
df = df.astype(object).where(pd.notnull(df), None)
data = df.to_dict(orient="records")

print(json.dumps(data, indent=4))

In [ ]:
# Upload data needs authentication headers
# Upload data needs data records in json query
res = req.post(table_api_url + "rows/new", json={"query": data}, headers=auth_headers)

# raise Exception if request fails
if not res.ok:
    raise Exception(res.text)

# TODO: maybe batches, append, not overwrite existing

# Upload metadata using the API

In [ ]:
# get metadata (from example file)
metadata = req.get("https://gist.githubusercontent.com/wingechr/228d76ea759e92feca53910794cba477/raw/ac95def814f385b85e8372194723e5fb81672796/oep_example1.metadata.json").json()

print(json.dumps(metadata, indent=4))

In [ ]:
# Upload metadata needs authentication headers
# Upload metadata needs metadata json
res = req.post(table_api_url + "meta/", json=metadata, headers=auth_headers)

# raise Exception if request fails
if not res.ok:
    raise Exception(res.text)

# Delete table using the API

In [ ]:
# Deleting tables needs authentication headers
res = req.delete(table_api_url, headers=auth_headers)

# raise Exception if request fails
if not res.ok:
    raise Exception(res.text)